In [1]:
import googlemaps
import pandas as pd

In [ ]:
API_KEY = 'GET-YOUR-KEY'

## Extracting the closest transit spots within 2000 meters

In [3]:
# Initialize the Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Example location
location = (33.7490, -84.3880)

results = gmaps.places_nearby(location=location, radius=2000, type='transit_station')
data = []

# Loop through each place and calculate driving distance
for place in results['results']:
    name = place['name']
    address = place.get('vicinity', 'Address not available')
    latitude = place['geometry']['location']['lat']
    longitude = place['geometry']['location']['lng']
    
    # Get directions from home_location to the transit stop
    directions = gmaps.directions(location, (latitude, longitude), mode="walking")
    
    # Get the distance from the directions response
    if directions:
        distance_meters = directions[0]['legs'][0]['distance']['value']
        distance_miles = distance_meters / 1609.34  # Convert meters to miles
    else:
        distance_miles = None
    
    data.append([name, address, latitude, longitude, distance_miles])

df = pd.DataFrame(data, columns=['Name', 'Address', 'Latitude', 'Longitude', 'Walking_Distance_miles'])
df.to_csv("transit_stops_with_walking_distance_miles.csv", index=False)


In [22]:
#5 different potential rental houses
houses = [
    {"id": 1, "location": (33.7490, -84.3880)}, 
    {"id": 2, "location": (34.0375, -84.5751)},
    {"id": 3, "location": (34.7530, -84.3870)}, 
    {"id": 4, "location": (33.7550, -84.3890)},
    {"id": 5, "location": (33.7570, -85.3860)} 
]

In [23]:
transit_stops = []
for house in houses:
    transits = []
    results = gmaps.places_nearby(location=house['location'], radius=2000, type='transit_station')
    for place in results['results']:
        name = place['name']
        latitude = place['geometry']['location']['lat']
        longitude = place['geometry']['location']['lng']
        transits.append({"name":name,"location":(latitude,longitude)})
    transit_stops.append(transits)
    

In [24]:
len(transit_stops[0])

20

## Best house using Average distance to transits 

In [25]:
def get_average_distance(house_location, transit_stops, mode="walking"):
    total_distance = 0
    for stop in transit_stops:
        directions = gmaps.directions(house_location, stop["location"], mode=mode)
        if directions:
            distance_meters = directions[0]['legs'][0]['distance']['value']
            total_distance += distance_meters / 1609.34
    return total_distance / len(transit_stops)

In [28]:
house_distances = []
for i, house in enumerate(houses):
    if len(transit_stops[i]) == 0:  # If there are no transit stops found
        avg_distance = None
    else:
        avg_distance = get_average_distance(house["location"], transit_stops[i])
    
    house_distances.append({"House_ID": house["id"], "avg_dist": avg_distance})

df_dist = pd.DataFrame(house_distances)
print(df_dist.head(20))

   House_ID  avg_dist
0         1  0.713429
1         2  0.694104
2         3       NaN
3         4  0.561938
4         5       NaN


In [31]:
best_house = df_dist.loc[df_dist['avg_dist'].idxmin()]
best_house

House_ID    4.000000
avg_dist    0.561938
Name: 3, dtype: float64

## Best house using Weighted Average distance to transits 

In [33]:
def get_weighted_distance(house_location, transit_stops, mode="walking"):
    total_weighted_distance = 0
    total_weight = 0
    for stop in transit_stops:
        directions = gmaps.directions(house_location, stop["location"], mode=mode)
        if directions:
            distance_meters = directions[0]['legs'][0]['distance']['value']
            distance_miles = distance_meters / 1609.34 
            weight = 1 / (distance_miles + 0.1)  # To avoid division by zero, adding a small constant
            total_weighted_distance += distance_miles * weight
            total_weight += weight
    return total_weighted_distance / total_weight if total_weight > 0 else None

In [37]:
house_distances = []
for i, house in enumerate(houses):
    if len(transit_stops[i]) == 0:  # If there are no transit stops found
        avg_distance = None
    else:
        avg_distance = get_weighted_distance(house["location"], transit_stops[i])
    
    house_distances.append({"House_ID": house["id"], "weight_avg_dist": avg_distance})

df_dist = pd.DataFrame(house_distances)
print(df_dist.head(20))

   House_ID  weight_avg_dist
0         1         0.533513
1         2         0.475245
2         3              NaN
3         4         0.437761
4         5              NaN


In [38]:
best_house = df_dist.loc[df_dist['weight_avg_dist'].idxmin()]
best_house

House_ID           4.000000
weight_avg_dist    0.437761
Name: 3, dtype: float64

## Best house using k-Nearest Neighbors to transits 

In [40]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [46]:
def get_total_distance(house_location, transit_stops, mode="walking"):
    distances = []
    for stop in transit_stops:
        directions = gmaps.directions(house_location, stop["location"], mode=mode)
        if directions:
            distance_meters = directions[0]['legs'][0]['distance']['value']
            distance_miles = distance_meters / 1609.34
            distances.append(distance_miles)
    return distances

In [84]:
house_distances = []
for i, house in enumerate(houses):
    if len(transit_stops[i]) == 0:  # If there are no transit stops found
        distances = None
    else:
        distances = get_total_distance(house["location"], transit_stops[i])
    if distances is not None:
        house_distances.append([i,distances])

In [90]:
final_prob_houses = [x[0] for x in house_distances]
final_prob_houses

[0, 1, 3]

In [91]:
final_house_distances = [x[1] for x in house_distances]
X = np.array(final_house_distances)

In [92]:
# K-NN (for K=3)
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X)
distances, indices = knn.kneighbors(X)

In [97]:
average_distances = distances.mean(axis=1)
best_house_idx = np.argmin(average_distances)
best_house = houses[final_prob_houses[best_house_idx]]
best_house

{'id': 4, 'location': (33.755, -84.389)}